In [ ]:
from tsim import Circuit
import numpy as np
from utils.encoder import ColorEncoder3

In [ ]:
# Define some parameters
# IMPORTANT: All rotation parameters are in units of pi

theta = 0.23
phi_0 = 0.45
phi_1 = 0.12
phi_2 = 0.34

In [ ]:
# We start by defininig a logical circuit (on 9 qubits), simply as a string in Stim format:
initialization_circuit = f"""
    R 0
    R 1  # arbitrary state prep
    R 2
    R_X({phi_2}) 2  # z_phi_2
    RX 3
    R_Z({2*theta}) 3  # y_2*theta
    RX 4
    R_Z({-2*theta}) 4  # y_-2*theta
    R 5
    R_X({phi_1}) 5  # z_phi_1
    RX 6
    R_Z({2*theta}) 6  # y_2*theta
    RX 7
    R_Z({-2*theta}) 7  # y_-2*theta
    R 8
    R_X({phi_0}) 8  # z_phi_0
"""

p = 0.01
body_circuit = f"""
    CNOT 2 0
    DEPOLARIZE2({p}) 2 0  # 2-qubit depolarizing noise after every 2-qubit gate
    CNOT 3 1
    DEPOLARIZE2({p}) 3 1
    CNOT 0 1
    DEPOLARIZE2({p}) 0 1
    CNOT 4 1
    DEPOLARIZE2({p}) 4 1
    CNOT 5 0
    DEPOLARIZE2({p}) 5 0
    CNOT 6 1
    DEPOLARIZE2({p}) 6 1
    CNOT 0 1
    DEPOLARIZE2({p}) 0 1
    CNOT 7 1
    DEPOLARIZE2({p}) 7 1
    CNOT 8 0
    DEPOLARIZE2({p}) 8 0
    X_ERROR({p}) 0 1 2 3 4 5 6 7 8  # bit-flip noise before measurements
    M 0 1 2 3 4 5 6 7 8
"""

for i in range(-9, 0):
    body_circuit += f"""
    DETECTOR rec[{i}]
    OBSERVABLE_INCLUDE({i + 9}) rec[{i}]
    """

c = Circuit(initialization_circuit + body_circuit)
c.diagram("timeline-svg", height=400)

In [ ]:
sampler = c.compile_sampler()
sampler

In [ ]:
sampler.sample(1000, batch_size=1000)

In [ ]:
# Here are some convenient tools to turn the logical circuit into a physical circuit:

encoder = ColorEncoder3()
# To use [17,1,5] code replace with ColorEncoder5() (from utils.codes import ColorEncoder5)

# provide the encoder with the logical initialization circuit.
encoder.initialize(initialization_circuit)

# now append the body circuit. The encoder will turn each gate into a transversal gate.
encoder.encode_transversally(body_circuit)

# retrieve the encoded circuit and plot it:
encoded_circuit = encoder.circuit
encoded_circuit.diagram("timeline-svg", height=600)

In [ ]:
print(
    f"""
Number of qubits: {encoded_circuit.num_qubits}
Number of measurements: {encoded_circuit.num_measurements}
Number of observables: {encoded_circuit.num_observables}
Number of detectors: {encoded_circuit.num_detectors}
"""
)

In [ ]:
sampler = encoded_circuit.compile_detector_sampler()

In [ ]:
detectors, observables = sampler.sample(
    100_000, batch_size=100_000, separate_observables=True
)

In [ ]:
import time

start = time.perf_counter()
sampler.sample(100_000, batch_size=100_000, separate_observables=True)
end = time.perf_counter()
print(f"Time per shot: {(end - start) / 100_000 * 1_000_000:.2f} μs")

In [ ]:
# We have 27 detectors (9 logical qubits * 3 stabilizers)
# We have sampled the detectors 100 times
detectors

In [ ]:
# We have 9 observables (9 logical qubits), sampled 100 times
observables

## Custom Encoding

In [ ]:
custom_encoding_circuit = f"""
    R 0 1 2 3 4 5
    TICK
    SQRT_Y_DAG 0 1 2 3 4 5
    DEPOLARIZE1({p}) 0 1
    TICK
    CZ 1 2 3 4 5 6
    DEPOLARIZE2({p}) 1 2
    TICK
    SQRT_Y 6
    DEPOLARIZE1({p}) 6
    TICK
    CZ 0 3 2 5 4 6
    TICK
    SQRT_Y 2 3 4 5 6
    DEPOLARIZE1({p}) 2 4 6
    TICK
    CZ 0 1 2 3 4 5
    TICK
    DEPOLARIZE1({p}) 0 1 2 3 4 5 6
    SQRT_Y 1 2 4
    X 3
"""

encoder = ColorEncoder3()
encoder.initialize(
    initialization_circuit, encoding_program_text=custom_encoding_circuit
)
encoder.encode_transversally(body_circuit)
encoded_circuit = encoder.circuit
encoded_circuit.diagram("timeline-svg", height=600)

## Noiseless Circuits

In [ ]:
# It is easy to remove noise from a circuit:
noiseless_circuit = encoded_circuit.without_noise()
noiseless_circuit.diagram("timeline-svg", height=600)

In [ ]:
sampler = noiseless_circuit.compile_detector_sampler()

In [ ]:
detectors, observables = sampler.sample(100, separate_observables=True)

In [ ]:
# Since there is not noise, there should not be any detection events
assert np.all(detectors == 0)

In [ ]:
observables